## Reinforecement Learning Models

### Imports and Utils

In [23]:
"""
Importing necessary libraries
"""
import pygame
import neat
import time
import os
import random

In [24]:
"""
Defining Utilities for the Game
"""
scale = 1.5
WIN_WIDTH = 288 * scale
WIN_HEIGHT = 512 * scale

def Load_Images(path, scale = 1):
    img = pygame.image.load(path)
    return pygame.transform.scale(img, (int(img.get_width() * scale), int(img.get_height() * scale)))

BIRD_IMGS = [Load_Images(os.path.join("Images", "bird1.png"), scale), Load_Images(os.path.join("Images", "bird2.png"), scale), Load_Images(os.path.join("Images", "bird3.png"), scale)]
PIPE_IMG = Load_Images(os.path.join("Images", "pipe.png"), scale)
BASE_IMG = Load_Images(os.path.join("Images", "base.png"), scale)
BG_IMG = Load_Images(os.path.join("Images", "bg.png"), scale)

In [25]:
"""
Defining Bird Class
"""
class Bird:
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25
    ROT_VEL = 10
    ANIMATION_TIME = 5

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.tilt = 0
        self.vel = 0
        self.cnt = 0
        self.img = self.IMGS[0]

    def jump(self):
        self.vel = -20
        self.height = self.y

    def move(self):
        s = min(16, self.vel + 1.5)
        self.vel = min(16, self.vel + 3)

        self.y = self.y + s
        if s < 0:
            if self.tilt < self.MAX_ROTATION:
                self.tilt += self.ROT_VEL
        else:
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL
    
    def draw(self, window):
        self.cnt += 1

        if self.cnt < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.cnt < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.cnt < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.cnt < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.cnt == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.cnt = 0

        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.cnt = self.ANIMATION_TIME*2

        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center=self.img.get_rect(topleft=(self.x, self.y)).center)
        window.blit(rotated_image, new_rect.topleft)

    def get_mask(self):
        return pygame.mask.from_surface(self.img)

In [26]:
"""
Defining Pipe Class
"""

class Pipe:
    GAP = 200
    VEL = 5

    def __init__(self, x):
        self.x = x
        self.height = 0

        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)
        self.PIPE_BOTTOM = PIPE_IMG

        self.passed = False
        self.set_height()

    def set_height(self):
        self.height = random.randrange(50, 400)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP

    def move(self):
        self.x -= self.VEL

    def draw(self, window):
        window.blit(self.PIPE_TOP, (self.x, self.top))
        window.blit(self.PIPE_BOTTOM, (self.x, self.bottom))

    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)

        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))

        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask, top_offset)

        if b_point or t_point:
            return True
        return False

In [27]:
"""
Defining Base Class
"""

class Base:
    VEL = 5
    WIDTH = BASE_IMG.get_width()
    IMG = BASE_IMG

    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH

    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH
        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, window):
        window.blit(self.IMG, (self.x1, self.y))
        window.blit(self.IMG, (self.x2, self.y))

In [28]:
"""
Game Structure and Functionality
"""
pygame.font.init()

class Game:

    def __init__(self, window, base, pipes, bird):
        self.window = window
        self.base = base
        self.pipes = pipes
        self.bird = bird

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        self.bird.jump()

            if not game_over:
                self.bird.move()
                for pipe in self.pipes:
                    pipe.move()
                    if pipe.collide(self.bird):
                        game_over = True
                    if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                        self.pipes.remove(pipe)
                    if not pipe.passed and pipe.x < self.bird.x:
                        pipe.passed = True
                        self.score += 1
                        self.pipes.append(Pipe(WIN_WIDTH + 50))
                self.base.move()

                if self.bird.y + self.bird.img.get_height() >= WIN_HEIGHT or self.bird.y < 0:
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display()
        pygame.quit()

    def display(self):
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (255, 255, 255))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [30]:
def main():
    bird = Bird(150, 300)
    pipes = [Pipe(300)]
    base = Base(650)
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = Game(window, base, pipes, bird)
    game.gameplay()

main()

### Model Creation and Training

### Testing and Plotting